In [2]:

import pandas as pd
from bokeh.io import output_notebook
output_notebook()
bokeh_width, bokeh_height = 500,400



Loading BokehJS ...

In [3]:
df = pd.read_csv('C:/Users/dasus/Documents/NCAS-M/NCAS/Data/depth_data/Mar-25-2022.csv')
df.head()

,Latitude,Longitude,Depth
0,18.099450,-67.190283,1.51
1,18.099433,-67.190267,1.51
2,18.099433,-67.190267,1.51
3,18.099433,-67.190267,1.51
4,18.099433,-67.190267,1.51


In [4]:
lat, lon= df.Latitude[0], df.Longitude[0]


In [5]:


import os 
api_key = os.environ['GOOGLE_API_KEY']



In [6]:


from bokeh.io import show
from bokeh.plotting import gmap
from bokeh.models import GMapOptions

def plot(lat, lng, zoom=16, map_type='roadmap'):
    gmap_options = GMapOptions(lat=lat, lng=lng, 
                               map_type=map_type, zoom=zoom)
    p = gmap(api_key, gmap_options, title='Bathymetry Map Parguera', 
             width=bokeh_width, height=bokeh_height)
    show(p)
    return p



In [7]:


p = plot(lat, lon)



In [8]:


def plot(lat, lng, zoom=16, map_type='roadmap'):
    gmap_options = GMapOptions(lat=lat, lng=lng, 
                               map_type=map_type, zoom=zoom)
    p = gmap(api_key, gmap_options, title='Bathymetry Map Parguera', 
             width=bokeh_width, height=bokeh_height)
    # beware, longitude is on the x axis ;-)
    center = p.circle([lng], [lat], size=10, alpha=0.5, color='red')
    show(p)
    return p

p = plot(lat, lon, map_type='terrain')



In [9]:


df.shape



(65, 3)

In [10]:


from bokeh.models import ColumnDataSource

def plot(lat, lng, zoom=19, map_type='roadmap'):
    gmap_options = GMapOptions(lat=lat, lng=lng, 
                               map_type=map_type, zoom=zoom)
    p = gmap(api_key, gmap_options, title='Bathymetry Map Parguera', 
             width=bokeh_width, height=bokeh_height)
    # definition of the column data source: 
    source = ColumnDataSource(df)
    # see how we specify the x and y columns as strings, 
    # and how to declare as a source the ColumnDataSource:
    center = p.circle('Longitude', 'Latitude', size=4, alpha=0.2, 
                      color='yellow', source=source)
    show(p)
    return p

p = plot(lat, lon, map_type='satellite')



In [11]:

import numpy as np
df['radius'] = np.sqrt(df['Depth'])/0.8
df.head()

,Latitude,Longitude,Depth,radius
0,18.099450,-67.190283,1.51,1.536026
1,18.099433,-67.190267,1.51,1.536026
2,18.099433,-67.190267,1.51,1.536026
3,18.099433,-67.190267,1.51,1.536026
4,18.099433,-67.190267,1.51,1.536026


In [12]:
from bokeh.models import HoverTool

def plot(lat, lng, zoom=19, map_type='roadmap'):
    gmap_options = GMapOptions(lat=lat, lng=lng, 
                               map_type=map_type, zoom=zoom)
    # the tools are defined below: 
    hover = HoverTool(
        tooltips = [
            # @price refers to the price column
            # in the ColumnDataSource. 
            ('Depth in Feet', '@Depth in Feet')
        ]
    )
    # below we replaced 'hover' (the default hover tool), 
    # by our custom hover tool
    p = gmap(api_key, gmap_options, title='Bathymetry Map Parguera', 
             width=bokeh_width, height=bokeh_height,
             tools=[hover, 'reset', 'wheel_zoom', 'pan'])
    source = ColumnDataSource(df)
    center = p.circle('Longitude', 'Latitude', size=4, alpha=0.5, 
                      color='yellow', source=source)
    show(p)
    return p

p = plot(lat, lon, map_type='satellite')

In [13]:
def plot(lat, lng, zoom=10, map_type='roadmap'):
    gmap_options = GMapOptions(lat=lat, lng=lng, 
                               map_type=map_type, zoom=zoom)
    hover = HoverTool(
        tooltips = [
            ('Depth in Feet', '@Depth')
        ]
    )
    p = gmap(api_key, gmap_options, title='Bathymetry Maps Parguera', 
             width=bokeh_width, height=bokeh_height,
             tools=[hover, 'reset', 'wheel_zoom', 'pan'])
    source = ColumnDataSource(df)
    # we use the radius column for the circle size: 
    center = p.circle('Longitude', 'Latitude', radius='radius', 
                      alpha=0.5, color='yellow', source=source)
    show(p)
    return p

p = plot(lat, lon, map_type='satellite', zoom=18)

In [16]:


from bokeh.transform import linear_cmap
from bokeh.small_palettes import RdYlBu as palette
from bokeh.models import ColorBar

# we are adding the dataframe as a parameter, 
# since we are now going to plot
# a different dataframe
def plot(df, lat, lng, zoom=10, map_type='roadmap'):
    gmap_options = GMapOptions(lat=lat, lng=lng, 
                               map_type=map_type, zoom=zoom)
    hover = HoverTool(
        tooltips = [
            ('Depth in Feet', '@Depth '),
            # the {0.} means that we don't want decimals
            # for 1 decimal, write {0.0}
        ]
    )
    p = gmap(api_key, gmap_options, title='Bathymetry Map Parguera', 
             width=bokeh_width, height=bokeh_height,
             tools=[hover, 'reset', 'wheel_zoom', 'pan'])
    source = ColumnDataSource(df)
    # defining a color mapper, that will map values of pricem2
    # between 2000 and 8000 on the color palette
    mapper = linear_cmap('Depth', palette, min(df.Depth), max(df.Depth))    
    # we use the mapper for the color of the circles
    center = p.circle('Longitude', 'Latitude', radius='radius', alpha=0.4, 
                      color=mapper, source=source)
    # and we add a color scale to see which values the colors 
    # correspond to 
    color_bar = ColorBar(color_mapper=mapper['transform'], 
                         location=(0,0))
    p.add_layout(color_bar, 'right')
    show(p)
    return p

p = plot(df, lat, lon, map_type='satellite', zoom=18)



ValueError: failed to validate LinearColorMapper(id='1799', ...).palette: expected an element of Seq(Color), got {3: ('#91bfdb', '#ffffbf', '#fc8d59'), 4: ('#2c7bb6', '#abd9e9', '#fdae61', '#d7191c'), 5: ('#2c7bb6', '#abd9e9', '#ffffbf', '#fdae61', '#d7191c'), 6: ('#4575b4', '#91bfdb', '#e0f3f8', '#fee090', '#fc8d59', '#d73027'), 7: ('#4575b4', '#91bfdb', '#e0f3f8', '#ffffbf', '#fee090', '#fc8d59', '#d73027'), 8: ('#4575b4', '#74add1', '#abd9e9', '#e0f3f8', '#fee090', '#fdae61', '#f46d43', '#d73027'), 9: ('#4575b4', '#74add1', '#abd9e9', '#e0f3f8', '#ffffbf', '#fee090', '#fdae61', '#f46d43', '#d73027'), 10: ('#313695', '#4575b4', '#74add1', '#abd9e9', '#e0f3f8', '#fee090', '#fdae61', '#f46d43', '#d73027', '#a50026'), 11: ('#313695', '#4575b4', '#74add1', '#abd9e9', '#e0f3f8', '#ffffbf', '#fee090', '#fdae61', '#f46d43', '#d73027', '#a50026')}